In [2]:
import tensorflow as tf
print("GPU available:", tf.config.list_physical_devices('GPU'))
print("Built with CUDA:", tf.test.is_built_with_cuda())

GPU available: []
Built with CUDA: True


In [3]:
import os
import numpy as np
import tensorflow as tf
from data_loader import DataGenerator
from sklearn.metrics import f1_score, precision_score, recall_score
import datetime
from helpers import f1 as f1_metric
from model import lstm_unet

def set_random_seeds(seed):
    """Set random seeds for reproducibility."""
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def create_data_generators():
    """Create data generators for training and validation."""
    train_gen = DataGenerator(
        image_dir="../Unet/data_v3_processed/train/images/images",
        mask_dir="../Unet/data_v3_processed/train/masks/masks",
        batch_size=8,
        time_steps=15,
        img_size=(256,256),
        normalize_images=True,
        normalize_masks=False,
        infinite=True,
    )
    
    val_gen = DataGenerator(
        image_dir="../Unet/data_v3_processed/val/images/images",
        mask_dir="../Unet/data_v3_processed/val/masks/masks",
        batch_size=8,
        time_steps=15,
        img_size=(256,256),
        normalize_images=True,
        normalize_masks=False,
        infinite=True
    )

    return train_gen, val_gen

def train_and_evaluate_model(run_number, train_generator, val_generator, 
                           epochs=3, steps_per_epoch=1000, validation_steps=5, 
                           random_seed=None):
    """Train a single model and return evaluation metrics."""
    print(f"\nTraining Run {run_number + 1}/5 (Seed: {random_seed})")
    print("=" * 50)
    
    if random_seed is not None:
        set_random_seeds(random_seed)
    
    model = lstm_unet()
    
    model.compile(optimizer='adam', 
                 loss='binary_crossentropy', 
                 metrics=[f1_metric])
    
    try:
        history = model.fit(train_generator,
                           validation_data=val_generator,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_steps=validation_steps,
                           verbose=1)

        model_path = os.path.join("models", f"lstm_unet_run{run_number + 1}.h5")

        os.makedirs("models", exist_ok=True)
        model.save(model_path)
        
        return model_path
        
    except Exception as e:
        print(f"ERROR in training: {e}")
        return None

# Main execution
random_seeds = [42, 123, 456, 789, 999]

for run in range(5):
    # CREATE FRESH GENERATORS FOR EACH RUN
    train_generator, val_generator = create_data_generators()
    
    model_path = train_and_evaluate_model(
        run, train_generator, val_generator, random_seed=random_seeds[run]
    )
    
    tf.keras.backend.clear_session()


Training Run 1/5 (Seed: 42)
Epoch 1/3
1000/1000 [==============================] - 1200s 1s/step - loss: 0.0551 - f1: 0.4645 - val_loss: 0.0110 - val_f1: 0.7017
Epoch 2/3
1000/1000 [==============================] - 1185s 1s/step - loss: 0.0042 - f1: 0.8976 - val_loss: 0.0102 - val_f1: 0.6724
Epoch 3/3
1000/1000 [==============================] - 1183s 1s/step - loss: 0.0023 - f1: 0.9157 - val_loss: 0.0077 - val_f1: 0.7606


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Training Run 2/5 (Seed: 123)
Epoch 1/3
1000/1000 [==============================] - 1191s 1s/step - loss: 0.0450 - f1: 0.6224 - val_loss: 0.0094 - val_f1: 0.7506
Epoch 2/3
1000/1000 [==============================] - 1183s 1s/step - loss: 0.0035 - f1: 0.9070 - val_loss: 0.0078 - val_f1: 0.7794
Epoch 3/3
1000/1000 [==============================] - 1181s 1s/step - loss: 0.0022 - f1: 0.9162 - val_loss: 0.0069 - val_f1: 0.7828

Training Run 3/5 (Seed: 456)
Epoch 1/3
1000/1000 [==============================] - 1191s 1s/step - loss: 0.0843 - f1: 4.4731e-04 - val_loss: 0.0171 - val_f1: 0.0000e+00
Epoch 2/3
1000/1000 [==============================] - 1183s 1s/step - loss: 0.0057 - f1: 0.3715 - val_loss: 0.0078 - val_f1: 0.7555
Epoch 3/3
1000/1000 [==============================] - 1183s 1s/step - loss: 0.0023 - f1: 0.9111 - val_loss: 0.0079 - val_f1: 0.7457

Training Run 4/5 (Seed: 789)
Epoch 1/3
1000/1000 [==============================] - 1191s 1s/step - loss: 0.0820 - f1: 0.2911 - val_l